<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/transformation_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка Textattack

In [1]:
!git clone https://github.com/QData/TextAttack.git


Cloning into 'TextAttack'...
remote: Enumerating objects: 21081, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 21081 (delta 314), reused 403 (delta 253), pack-reused 20535
Receiving objects: 100% (21081/21081), 25.85 MiB | 6.77 MiB/s, done.
Resolving deltas: 100% (15623/15623), done.


In [6]:
%cd /content/TextAttack
!pip install -e .

/content/TextAttack
Obtaining file:///content/TextAttack
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11


Создание класса RocketWordSwap
(вместо Rocket может быть использовано любое другое слово)

In [8]:
from textattack.transformations import WordSwap
class RocketWordSwap(WordSwap):

    def _get_replacement_words(self, word):
    #Вернет 'Rocket', в не зависимости от того какое 'слово' было в оригинальном тексте.
            return ['Rocket']

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Импорт модели

In [9]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
''' вместо "bert-base-uncased-ag-news" можно выбрать одну из 82 моделей на сайте https://huggingface.co/textattack
'''
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# создание целевой функции с помошью модели
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# импорт набора данных
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


In [10]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

# Применение класса RocketWordSwap в качестве атаки преобразования.
transformation = RocketWordSwap()
# Применение ограничений модификации уже изменненных индексов и стоп-слов.
constraints = [RepeatModification(),
               StopwordModification()]
# Применение метода жадного поиска (доступные методы поиска представлены на сайте https://textattack.readthedocs.io/en/latest/apidoc/textattack.search_methods.html#search-methods)
search_method = GreedySearch()
# непосредственно атака.
attack = Attack(goal_function, constraints, transformation, search_method)

Вывод атаки

In [11]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  RocketWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


Вывод модели данных

In [12]:
print(dataset[0])

(OrderedDict([('text', "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")]), 2)


In [ ]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

#The following legacy tutorial code shows how the Attack API works in detail.

#logger = CSVLogger(color_method='html')

#num_successes = 0
#i = 0
#while num_successes < 10:
    #result = next(results_iterable)
#    example, ground_truth_output = dataset[i]
#    i += 1
#    result = attack.attack(example, ground_truth_output)
#    if isinstance(result, SuccessfulAttackResult):
#        logger.log_attack_result(result)
#        num_successes += 1
#       print(f'{num_successes} of 10 successes complete.')

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  RocketWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



 10%|█         | 1/10 [04:39<41:51, 279.11s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [04:39<41:59, 279.99s/it]

[[Business (100%)]] --> [[Sci/tech (100%)]]

Fears for T N [[pension]] after [[talks]] [[Unions]] [[representing]] workers at Turner   Newall say they are 'disappointed' after [[talks]] with stricken [[parent]] firm [[Federal]] [[Mogul]].

Fears for T N [[Rocket]] after [[Rocket]] [[Rocket]] [[Rocket]] workers at Turner   Newall say they are 'disappointed' after [[Rocket]] with stricken [[Rocket]] firm [[Rocket]] [[Rocket]].




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [49:04<3:16:16, 1472.00s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.


